<a href="https://colab.research.google.com/github/KhanhDuy211/TH_TTNT_Tuan3/blob/main/Minimax%26Alpha_Beta_Tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# --- THUẬT GIẢI MINIMAX ÁP DỤNG VÀ BÀI TOÁN TICTACTOE --- #

import copy
import math
import random
import numpy

X = "X"
O = "O"
EMPTY = None
user = None
ai = None

def initial_state():
  """
  Returns starting state of the board.
  """
  return [[EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY]]

def player(board):
  """
  Returns player (X or O) who has the next turn on a board.
  """
  x_count = sum(row.count(X) for row in board)
  o_count = sum(row.count(O) for row in board)

  if x_count == o_count:
    return X # X always starts, so if counts are equal, it's X's turn
  else:
    return O # If X has made more moves, it's O's turn

def actions(board):
  """
  Returns set of all possible actions (i, j) available on the board.
  """
  res = set()
  board_len = len(board)
  for i in range(board_len):
    for j in range(board_len):
      if board[i][j] == EMPTY:
        res.add((i,j))
  return res

def result(board, action):
  """
  Returns the board that results from making move (i, j) on the board.
  """
  curr_player = player(board)
  result_board = copy.deepcopy(board)
  (i,j) = action
  result_board[i][j] = curr_player
  return result_board

def get_horizontal_winner(board):
  winner_val = None
  board_len = len(board)
  for i in range(board_len):
    if board[i][0] != EMPTY: # Only check if the first cell is not empty
      winner_val = board[i][0]
      for j in range(board_len): # Iterate through the row
        if board[i][j] != winner_val: # If any cell in the row is different, it's not a winner
          winner_val = None
          break # No need to check further in this row
    if winner_val: # If a winner was found in this row, return it
      return winner_val
  return None # No horizontal winner found

def get_vertical_winner(board):
  winner_val = None
  board_len = len(board)
  for j in range(board_len):
    if board[0][j] != EMPTY: # Only check if the first cell is not empty
      winner_val = board[0][j]
      for i in range(board_len): # Iterate through the column
        if board[i][j] != winner_val: # If any cell in the column is different, it's not a winner
          winner_val = None
          break # No need to check further in this column
    if winner_val: # If a winner was found in this column, return it
      return winner_val
  return None # No vertical winner found

def get_diagonal_winner(board):
  winner_val = None
  board_len = len(board)

  # Check main diagonal
  if board[0][0] != EMPTY:
    winner_val = board[0][0]
    for i in range(board_len):
      if board[i][i] != winner_val:
        winner_val = None
        break
  if winner_val:
    return winner_val

  # Check anti-diagonal
  if board[0][board_len - 1] != EMPTY:
    winner_val = board[0][board_len - 1]
    for i in range(board_len):
      j = board_len - 1 - i
      if board[i][j] != winner_val:
        winner_val = None
        break
  return winner_val # Will be None if no diagonal winner

def winner(board):
  """
  Returns the winner of the game, if there is one.
  """
  winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board)
  return winner_val

def terminal(board):
  """
  Returns True if game is over, False otherwise.
  """
  if winner(board) != None:
    return True
  for i in board:
    for j in i:
      if j == EMPTY:
        return False
  return True

def utility(board):
  """
  Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
  """
  winner_val = winner(board)
  if winner_val == X:
    return 1
  elif winner_val == O: # Corrected from 0 to O for the player
    return -1
  return 0

def maxValue(state):
  if terminal(state):
    return utility(state)
  v = -math.inf # Changed from math.inf to -math.inf for max initialization
  for action in actions(state):
    v = max(v, minValue(result(state, action)))
  return v

def minValue(state):
  if terminal(state):
    return utility(state)
  v = math.inf
  for action in actions(state):
    v = min(v, maxValue(result(state, action)))
  return v

def minimax(board):
  """
  Returns the optimal action for the current player on the board.
  """
  current_player = player(board)

  if current_player == ai: # It's AI's turn
    best_action = None
    if ai == X: # AI is X, so it wants to maximize
      best_val = -math.inf
      for action in actions(board):
        val = minValue(result(board, action))
        if val > best_val:
          best_val = val
          best_action = action
    else: # ai == O, so it wants to minimize
      best_val = math.inf
      for action in actions(board):
        val = maxValue(result(board, action))
        if val < best_val:
          best_val = val
          best_action = action
    return best_action
  else:
    return None # Should not reach here during AI's turn calculation

if __name__ == "__main__":
  board = initial_state()
  print("Choose a player (X/O):")
  user_choice = input().upper() # Store user's choice in a different variable to avoid conflict with global 'user'

  if user_choice == "X":
    user = X
    ai = O
  else:
    user = O
    ai = X

  print(f"You are {user}, AI is {ai}.")

  while True:
    game_over = terminal(board)
    current_player_turn = player(board)

    if game_over:
      winner_player = winner(board)
      if winner_player is None:
        print("Game Over: Tie.")
      else:
        print(f"Game Over: {winner_player} wins.")
      break
    else:
      if current_player_turn == ai: # It's AI's turn
        print("AI is thinking...")
        move = minimax(board)
        if move:
            board = result(board, move)
            print("AI made a move:")
        else:
            print("AI could not find a move. This should not happen in a non-terminal state.")
        print(numpy.array(board))

      elif current_player_turn == user: # It's User's turn
        print("Enter the position to move (row,col, 0-indexed):")
        while True:
          try:
            i = int(input("Row:"))
            j = int(input("Col:"))
            if (0 <= i < 3) and (0 <= j < 3) and board[i][j] == EMPTY:
              board = result(board, (i,j))
              print(numpy.array(board))
              break
            else:
              print("Invalid move. Position out of bounds or already taken. Try again.")
          except ValueError:
            print("Invalid input. Please enter a number for row and column.")


print("----------------------------------------------------------------------------------------------------------------------")
# --- THUẬT TOÁN ALPHA-BETA ÁP DỤNG VÀ BÀI TOÁN TICTACTOE --- #
import os, math
def GetWinner(board):
  """
  Returns the winner in the current board if there is one, otherwise it returns None.
  """

  if board[0] == board[1] and board[1] == board[2]:
    return board[0]
  elif board[3] == board[4] and board[4] == board[5]:
    return board[3]
  elif board[6] == board[7] and board[7] == board[8]:
    return board[6]
# vertical
  elif board[0] == board[3] and board[3] == board[6]:
    return board[0]
  elif board[1] == board[4] and board[4] == board[7]:
    return board[1]
  elif board[2] == board[5] and board[5] == board[8]:
    return board[2]
# diagonal
  elif board[0] == board[4] and board[4] == board[8]:
    return board[0]
  elif board[2] == board[4] and board[4] == board[6]:
    return board[2]
def PrintBoard(board):
  """
  Clears the console and prints the current board.
  """
  os.system('cls' if os.name=='nt' else 'clear')
  print(f'''
{board[0]}|{board[1]}|{board[2]}
{board[3]}|{board[4]}|{board[5]}
{board[6]}|{board[7]}|{board[8]}
''')
def GetAvailableCells(board):
  """
  Returns a list of indices containing all available cells in a board.
  """
  available = list()
  for cell in board:
    if cell != "X" and cell != "O":
      available.append(cell)
  return available


def minimax(position, depth, alpha, beta, isMaximizing):
  """
  The AI algorithm responsible for choosing the best move. Returns best value of a move.
  """
# evaluate current board: if maximizing player won -> return 10
# if minimizing player won -> return -10
# if no one is winning (tie) -> return 0
# NOTE: Even though the following AI plays perfectly, it might
# choose to make a move which will result in a slower victory
# or a faster loss. Lets take an example and explain it
# Assume that there are 2 possible ways for X to win the game from a give board state.
# Move A : X can win in 2 move
# Move B : X can win in 4 moves
# Our evaluation will return a value of +10 for both moves A and B. Even though the move A
# is better because it ensures a faster victory, our AI may choose B sometimes. To overcome
# this problem we subtract the depth value from the evaluated score. This means that in case
# of a victory it will choose a the victory which takes least number of moves and in case of
# a loss it will try to prolong the game and play as many moves as possible.
  winner = GetWinner(position)
  if winner != None:
    return 10 - depth if winner == "X" else -10 + depth
  if len(GetAvailableCells(position)) == 0:
    return 0
  if isMaximizing:
    maxEval = -math.inf
    for cell in GetAvailableCells(position):
      position[cell - 1] = "X"
      Eval = minimax(position, depth + 1, alpha, beta, False)
      maxEval = max(maxEval, Eval)
      alpha = max(alpha, Eval)
      position[cell - 1] = cell
      if beta <= alpha:
        break
    return maxEval
  else:
      minEval = +math.inf
      for cell in GetAvailableCells(position):
        position[cell - 1] = "O"
        Eval = minimax(position, depth + 1, alpha, beta, True)
        minEval = min(minEval, Eval)
        beta = min(beta, Eval)
        position[cell - 1] = cell
        if beta <= alpha:
          break # prune
      return minEval
def FindBestMove(currentPosition, AI):
  """
  This will return the best possible move for the player.
  Will Traverse all cells, evaluate minimax function for all empty cells.
  And return the cell with optimal value.
  Parameters:
    currentPosition (list): The current board to find best move for.
    AI (str): The AI Player ("X" or "O").
  Returns:
  int: Index of best move for the current position.
  """
  bestVal = -math.inf if AI == "X" else +math.inf
  bestMove = -1
  for cell in GetAvailableCells(currentPosition):
    currentPosition[cell - 1] = AI
    moveVal = minimax(currentPosition, 0, -math.inf, +math.inf, False if AI == "X" else True)
    currentPosition[cell - 1] = cell
    if (AI == "X" and moveVal > bestVal):
      bestMove = cell
      bestVal = moveVal
    elif (AI == "O" and moveVal < bestVal):
      bestMove = cell
      bestVal = moveVal
  return bestMove
def main():
  player = input("Play as X or O? ").strip().upper()
  AI = "O" if player == "X" else "X";
  currentGame = [*range(1, 10)]
# X always starts first.
  currentTurn = "X";
  counter = 0
  while True:
    if currentTurn == AI:
# NOTE: if the AI starts first, it'll always choose index 0 so to save time you could play it.
      cell = FindBestMove(currentGame, AI)
      currentGame[cell - 1] = AI
      currentTurn = player
    elif currentTurn == player:
      PrintBoard(currentGame)
      while True:
        humanInput = int(input("Enter Number:").strip())
        if humanInput in currentGame:
          currentGame[humanInput - 1] = player
          currentTurn = AI
          break
        else:
          PrintBoard(currentGame)
          print("Cell Not Available.")
    if GetWinner(currentGame) != None:
        PrintBoard(currentGame)
        print(f"{GetWinner(currentGame)} WON!!!")
        break
    counter += 1
    if GetWinner(currentGame) == None and counter == 9:
      PrintBoard(currentGame)
      print("Tie.")
      break
if __name__ == "__main__":
  main()

Choose a player (X/O):
X
You are X, AI is O.
Enter the position to move (row,col, 0-indexed):
Row:1
Col:1
[[None None None]
 [None 'X' None]
 [None None None]]
AI is thinking...
AI made a move:
[['O' None None]
 [None 'X' None]
 [None None None]]
Enter the position to move (row,col, 0-indexed):
Row:2
Col:2
[['O' None None]
 [None 'X' None]
 [None None 'X']]
AI is thinking...
AI made a move:
[['O' None None]
 [None 'X' None]
 ['O' None 'X']]
Enter the position to move (row,col, 0-indexed):
Row:2
Col:1
[['O' None None]
 [None 'X' None]
 ['O' 'X' 'X']]
AI is thinking...
AI made a move:
[['O' 'O' None]
 [None 'X' None]
 ['O' 'X' 'X']]
Enter the position to move (row,col, 0-indexed):
Row:1
Col:2
[['O' 'O' None]
 [None 'X' 'X']
 ['O' 'X' 'X']]
AI is thinking...
AI made a move:
[['O' 'O' None]
 ['O' 'X' 'X']
 ['O' 'X' 'X']]
Game Over: O wins.
----------------------------------------------------------------------------------------------------------------------
Play as X or O? x

1|2|3
4|5|6
7|

CHƯƠNG 1 : THUẬT TOÁN MINIMAX

1.1.
